[컨볼루션 신경망 모델 만들어보기](https://tykimos.github.io/2017/03/08/CNN_Getting_Started/)  
하기 내용은 상기 링크를 정리한것.

## 문제 정의하기
손그림인 삼각형, 사각형, 원을 그려 이미지로 저장한 다음 이른 분류하는 모델을 생성한다.  
문제 형태와 입출력을 다음과 같이 정의한다.  
* 문제 형태 : 다중 클래스 분류
* 입력 : 손으로 그린 삼각형, 사각형, 원 이미지
* 출력 : 삼각형, 사각형, 원일 확률을 나타내는 벡터  
  
가장 처음 필요한 패키지를 불러오고, 매번 실행 시마다 결과가 달라지지 않도록 랜덤 시드를 명시적으로 지정한다.  


In [11]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(3)

## 데이터 준비하기
손으로 그린 삼각형, 사각형 원 이미지를 만들기 위해서는 여러가지 방법이 있을수 있다.  
테블릿을 이용할 수도 있고, 종이에 그려 사진으로 찍을 수 있다.  
그림 이미지 사이즈는 그리는 툴로 $24 \times 24$ 정도로 해봤다.
![](http://tykimos.github.io/warehouse/2017-3-8_CNN_Getting_Started_1.png)
모양별로 20개 정도를 만들어서 15개를 훈련에 사용하고, 5개를 테스트에 사용해본다.  
이미지는 png나 jpg로 저장한다.  
실제로 데이터셋이 어떻게 구성되어 있는지 모른 체 튜토리얼을 따라하거나,  
예제 코드를 실행시키다보면 결과는 잘 나오지만 막상 실제 문제에 적용할때는 막막할 때가 많다.  
간단한 예제로 직접 데이터셋을 만들어봄으로써 실제 문제에 접근할 때 시행착오를 줄이는 것이 중요하다.  

데이터 폴더는 다음과 같이 구성한다.

* train
    * circle
    * rectangle
    * triangle
* test
    * circle
    * rectangle
    * traingle  
      
![](http://tykimos.github.io/warehouse/2017-3-8_CNN_Getting_Started_2.png)  
아래의 링크를 이용하여 다운로드가 가능하다.  
http://tykimos.github.io/warehouse/2017-3-8_CNN_Getting_Started_handwriting_shape.zip

## 데이터셋 생성하기
케라스에서는 이미지 파일을 쉽게 학습시킬 수 있도록 ImangeDataGenerator 클래스를 제공한다.  
ImageDataGenerator클래스는 데이터 증강 (data augmentation)을 위해 막강한 기능을 제공하는데,  
이 기능들은 다른 곳에서 다루고,  
이 곳에서는 특정 폴더에 이미지를 분류 해 놓았을 때 이를 학습시키기 위한 데이터셋으로 만들어주는 기능을 사용해보겠다.    
먼저 ImageDataGenerator클래스를 이용하여 객체를 생성한 뒤  
flow_from_directory() 함수를 호출하여 제네레이터(generator)를 생성한다.  
flow_from_directory() 함수의 주요인자는 다음과 같다.  
* 첫번째 인자 : 이미지 경로를 지정한다.
* target_size : 패치 이미지 크기를 지정한다.  
폴더에 있는 원본 이미지 크기가 다르더라도 target_size에 지정된 크기로 자동 조절된다.
* batch_size : 배치 크기를 지정한다.
* class_mode : 분류 방식에 대해서 지정한다. 
    * categorical : 2D one-hot 부호화된 라벨이 반환된다.
    * binary : 1D 이진 라벨이 반환된다. 
    * sparse : 1D 정수 라벨이 반환된다.
    * None : 라벨이 반환되지 않는다.  

본 예제에서는 패치 이미지 크기를 $24 \times 24$로 하였으니 target_size도 (24, 24)로 셋팅하였다.  
훈련 데이터 수가 클래스당 15개이니   
배치 크기를 3으로 지정하여(가중치 업데이트)   
총 5번 배치를 수행하면 하나의 epoch가 수행될 수 있도록 하였다.   
다중 클래스 문제이므로 class_mode는 'categorical'로 지정하였다.  
그리고 제네레이터는 훈련용과 검증용으로 두 개를 만들었다.  

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    './handwriting_shape/train',
    target_size=(24, 24),
    batch_size=3,
    class_mode='categorical'
)

Found 45 images belonging to 3 classes.


In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    './handwriting_shape/test',
    target_size=(24, 24),
    batch_size=3, 
    class_mode='categorical'
)

Found 15 images belonging to 3 classes.


## 모델 구성하기
영상 분류에 높은 성능을 보이고 있는 컨볼루션 신경망 모델을 구성해본다.  
각 레이어들은 이전 강좌에서 살표보았으므로 크게 어려움없이 구성할 수 있다.  
    * 컨볼루션 레이어 : 입력 이미지 크기 $24 \times 24$, 입력 이미지 채널 3개, 필터 크기 $3 \times 3$, 필터 수 32개, 활성화 함수 'relu'
    * 컨볼루션 레이어 : 필터 크기 $3 \times 3$, 필터 수 64개, 활성화 함수 'relu'
    * 맥스풀링 레이어 : 풀 크기 $2 \times 2$
    * 플래튼 레이어(벡터화)
    * 댄스 레이어 : 출력 뉴런 수 128개, 활성화 함수 'relu'
    * 댄스 레이어 : 출력 뉴런 수 3개, 활성화 함수 'sotfmax'

In [12]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(24, 24, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

구성화한 모델을 가시화하면 아래와 같다.

In [14]:
from Ipython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

ModuleNotFoundError: No module named 'Ipython'

## 모델 학습과정 설정하기
모델을 정의했다면 모델을 손실함수와 최적화 알고리즘으로 엮는다.  
* loss : 현재 가중치 세트를 평가하는데 사용한 손실 함수이다.  
다중 클래스 문제이므로 'categorical_crossentropy'으로 지정한다.  
* optimizer : 최적의 가중치를 검색하는 데 사용되는 최적화 알고리즘으로 효율적인 경사 하강법 알고리즘 중 하나인 'adam'을 사용한다.  
* metrics : 평가 척도를 나타내며 분류 문제에서는 일반적으로 'accuracy'으로 지정한다.  

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## 모델 학습시키기
케라스에서는 모델을 학습시킬 때 주로 fit() 함수를 사용하지만 제네레이터로 생성 된 배치로 학습시킬 경우에는 fit_generator()함수를 사용한다.  
본 예제는 ImageDataGenerator라는 제네레이터로 이미지를 담고 있는 배치로 학습시키기 때문에 fit_generator()함수를 사용한다.    
* 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정한다.  
본 예제에서는 앞서 생성한 train_generator으로 지정한다.  <br>
<br>
* steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정한다.  
총 45개의 훈련 샘플이 있고 배치 사이즈가 3이므로 15 스텝으로 지정한다.  <br>
<br>
* epochs: 전체 훈련 데이터 셋에 대해 학습 반복 횟수를 지정한다.  
100번을 반복적으로 학습시켜본다.  <br>
<br>

* validation_data : 검증데이터셋을 제공할 제네레이터를 지정한다.  
본 예제에서는 앞서 생성한 validation_generator으로 지정한다.  <br>
<br>
* validation_steps :  한 epoch 종료 시 마다 검증할 떄 사용되는 검증 스텝 수를 지정한다.  
총 15개의 검증 샘플(test 이미지 개수)이 있고 배치사이즈가 3이므로 5스텝으로 지정한다.  <br>
<br>


In [16]:
model.fit_generator(
    train_generator,
    steps_per_epoch=15,
    epochs=50,
    validation_data=test_generator,
    validation_steps=5
)

Epoch 1/50
15/15 [==============================] - 120s 8s/step - loss: 1.1965 - acc: 0.5556 - val_loss: 0.7390 - val_acc: 0.6667
Epoch 2/50
15/15 [==============================] - 0s 17ms/step - loss: 0.5381 - acc: 0.7556 - val_loss: 0.3682 - val_acc: 0.9333
Epoch 3/50
15/15 [==============================] - 0s 18ms/step - loss: 0.1749 - acc: 0.9556 - val_loss: 0.1652 - val_acc: 1.0000
Epoch 4/50
15/15 [==============================] - 0s 19ms/step - loss: 0.0503 - acc: 0.9778 - val_loss: 0.1048 - val_acc: 0.9333
Epoch 5/50
15/15 [==============================] - 0s 20ms/step - loss: 0.0105 - acc: 1.0000 - val_loss: 0.0509 - val_acc: 1.0000
Epoch 6/50
15/15 [==============================] - 0s 19ms/step - loss: 0.0027 - acc: 1.0000 - val_loss: 0.0367 - val_acc: 1.0000
Epoch 7/50
15/15 [==============================] - 0s 18ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0450 - val_acc: 1.0000
Epoch 8/50
15/15 [==============================] - 0s 18ms/step - loss: 7.0594e-04

## 모델 평가하기
학습한 모델을 평가해본다.  
제네레이터에서 제공되는 샘플로 평가할 때는 evaluate_generator함수를 사용한다.

In [18]:
print("--- Evaluate ---")
scores = model.evaluate_generator(test_generator, steps=5)
print("{}: {}%".format(model.metrics_names[1], scores[1]*100))

--- Evaluate ---
acc: 100.0%


데이터셋이 적고 간단한 모델임에도 100%라는 높은 정확도를 얻었다.  

## 모델 사용하기
모델 사용 시에 제네레이터에서 제공되는 샘플을 입력할 때는 predict_generator함수를 사용한다.  
예측 결과는 클래스별 확률 벡터로 출력되며,  
클래스에 해당하는 열을 알기 위해서는 제네레이터의 'class_indices'를 출력하면 해당 열의 클래스 명을 알려준다.  

In [22]:
print("--- Predict ---")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

--- Predict ---
{'circle': 0, 'rectangle': 1, 'triangle': 2}
[[0.000 0.000 1.000]
 [1.000 0.000 0.000]
 [0.000 1.000 0.000]
 [0.000 1.000 0.000]
 [0.000 1.000 0.000]
 [0.099 0.889 0.012]
 [0.000 1.000 0.000]
 [0.987 0.000 0.013]
 [0.000 0.000 1.000]
 [1.000 0.000 0.000]
 [1.000 0.000 0.000]
 [0.000 0.000 1.000]
 [0.000 0.001 0.999]
 [1.000 0.000 0.000]
 [0.000 0.000 1.000]]


## 전체 소스

In [24]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(3)

In [27]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    './handwriting_shape/train',
    target_size=(24, 24),
    batch_size=3,
    class_mode='categorical')

Found 45 images belonging to 3 classes.


In [29]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    './handwriting_shape/test',
    target_size=(24, 24),
    batch_size=3,
    class_mode='categorical'
)

Found 15 images belonging to 3 classes.


In [32]:
#2. 모델 구성하기
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), 
                activation='relu',
                input_shape=(24, 24, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [33]:
# 3. 모델학습과정 설정하기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
# 4. 모델 학습시키기
model.fit_generator(
    train_generator,
    steps_per_epoch=15,
    epochs=50,
    validation_data=test_generator,
    validation_steps=5)

Epoch 1/50
15/15 [==============================] - 1s 44ms/step - loss: 0.8092 - acc: 0.6889 - val_loss: 0.3883 - val_acc: 0.9333
Epoch 2/50
15/15 [==============================] - 0s 18ms/step - loss: 0.1185 - acc: 0.9778 - val_loss: 0.0374 - val_acc: 1.0000
Epoch 3/50
15/15 [==============================] - 0s 19ms/step - loss: 0.0076 - acc: 1.0000 - val_loss: 0.0386 - val_acc: 1.0000
Epoch 4/50
15/15 [==============================] - 0s 18ms/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.0340 - val_acc: 1.0000
Epoch 5/50
15/15 [==============================] - 0s 18ms/step - loss: 5.2191e-04 - acc: 1.0000 - val_loss: 0.0119 - val_acc: 1.0000
Epoch 6/50
15/15 [==============================] - 0s 20ms/step - loss: 2.4511e-04 - acc: 1.0000 - val_loss: 0.0144 - val_acc: 1.0000
Epoch 7/50
15/15 [==============================] - 0s 19ms/step - loss: 1.5563e-04 - acc: 1.0000 - val_loss: 0.0086 - val_acc: 1.0000
Epoch 8/50
15/15 [==============================] - 0s 21ms/step - loss

In [37]:
# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("{}: {}%".format(model.metrics_names[1], scores[1]*100))

-- Evaluate --
acc: 100.0%


In [39]:
# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x : "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
{'circle': 0, 'rectangle': 1, 'triangle': 2}
[[0.000 0.000 1.000]
 [1.000 0.000 0.000]
 [0.000 0.001 0.999]
 [0.000 1.000 0.000]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.075 0.916 0.009]
 [0.000 1.000 0.000]
 [0.000 1.000 0.000]
 [0.000 0.000 1.000]
 [1.000 0.000 0.000]
 [0.999 0.000 0.001]
 [1.000 0.000 0.000]
 [0.000 1.000 0.000]
 [1.000 0.000 0.000]]


## 요약
이미지 분류 문제에 높은 성능을 보이고 있는 컨볼루션 신경망 모델을 이용하여 직접 만든 데이터 셋으로 학습 평가를 해보았다.  
학습결과는 좋게 나왔지만 이 모델은 한사람이 그린 것에 대해서만 학습이 되어 다른 사람이 그린 모양은 분류를 잘못한다.  
이를 해결하기 위한 방안으로 '데이터 부풀리기' 기법이있다.  <br>
<br>
참고로 실제 문제에 적용하기 전에 데이터셋을 직접 만들어보거나 좀더 쉬운 문제로 추상화해서 프로토 타이핑 하는 것을 추천한다.  
객담도말된 결핵 이미지를 판별하는 모델을 만들 때,  
결핵 이미지를 바로 사용하지 않고,  
MNIST의 손글씨 중 '1', '7'을 결핵이라고 보고, 나머지는 결핵이 아닌 것으로 학습시켜봤었다.  
결핵균이 간균(막대모양)이라 적절한 프로토 타이핑이었고,  
타이핑 모델과 실제 데이터셋으로 학습한 모델 결과가 크게 차이나지 않는다.
